The usual magic stuff

In [1]:
import pythran
%load_ext pythran.magic

Test basic types

In [2]:
%%pythran
# simple types
#pythran export identity(int)
#pythran export identity(None)
#pythran export identity(str)

# parametric types
#pythran export identity(int list)
#pythran export identity(int set)
#pythran export identity(int:str dict)
#pythran export identity((int, int, str))

# numpy stuff
#pythran export identity(int[])
#pythran export identity(int[:,:])
#pythran export identity(int[][][])

def identity(x):
    return x

In [3]:
for elem in (int, str, list, set, dict):
    assert isinstance(identity(elem()), elem), elem

In [4]:
assert identity(None) is None
assert isinstance(identity((1,1,"1")), tuple)

Numpy arrays keep the same id when passed through pythran. this is not guaranteed for other containers

In [5]:
import numpy
x = numpy.ones(1, dtype=int)
assert x is identity(x)

y = numpy.ones((1, 1), dtype=int)
assert y is identity(y)

z = numpy.ones((1, 1, 1), dtype=int)
assert z is identity(z)

In [6]:
%%pythran
#pythran export inplace_modification(int list)
def inplace_modification(l):
    l[0] = 0
    return l

Exporting function 'inplace_modification' that modifies its List argument. Beware that this argument won't be modified at Python call site


In [7]:
l = [1, 2, 3]
lp = inplace_modification(l)
assert l is not lp
assert l != lp

It's possible to declare the overloads in a single export using the ``or`` keyword

In [8]:
%%pythran
#pythran export strint(str or int, str or int)
def strint(x, y):
    return y, x

In [9]:
strint(1, 2)

(2, 1)

In [10]:
strint('1', '2')

('2', '1')

In [11]:
strint(1, '2')

('2', 1)

In [12]:
strint('1', 2)

(2, '1')

The ``or`` operator also works inside polymorphic types, but it has lower precedence than ``set``, ``dict`` etc.

In [13]:
%%pythran
#pythran export set_of(int or str set)
def set_of(x): return x

In [14]:
set_of(1)

1

In [15]:
set_of({'1'})

{'1'}

Use  ``[ ]``  to force ordering

In [16]:
%%pythran
#pythran export set_of([int or str] set)
def set_of(x): return x

In [17]:
set_of({1})

{1}

In [18]:
set_of({'1'})

{'1'}

Overload for different scalar types are most of the time not ambiguous:

In [19]:
%%pythran
#pythran export scalar(bool)
#pythran export scalar(int)
#pythran export scalar(float)
#pythran export scalar(complex)
def scalar(x): return str(x)

In [20]:
print(scalar(True))
print(scalar(1))
print(scalar(1.1))
print(scalar(1.1+0j))

True
1
1.1
(1.1,0)


It works fine for scalars of differents size / sign

In [21]:
%%pythran
#pythran export dtype(complex64)
#pythran export dtype(complex128)
#pythran export dtype(complex256)
import numpy as np
def dtype(x): return x.real, x.imag

In [22]:
import numpy as np
x64r, x64i = dtype(np.complex64(1.5 + -1.5j))
print(x64r, x64i)
x128r, x128i = dtype(np.complex128(1.5 + -1.5j))
print(x128r, x128i)
x256r, x264i = dtype(np.complex256(1.5 + -1.5j))
print(x256r, x264i)

1.5 -1.5
1.5 -1.5
1.5 -1.5


It also works correctly for ndarray of different dimension and dtype:

In [23]:
%%pythran
#pythran export array(int8[])
#pythran export array(int16[][])
#pythran export array(int16[][][])
import numpy
def array(x): return x.shape, x.itemsize

In [24]:
import numpy as np
print(array(np.array([1], dtype=np.int8)))
print(array(np.array([[1]], dtype=np.int16)))
print(array(np.array([[[1]]], dtype=np.int16)))

((1,), 1)
((1, 1), 2)
((1, 1, 1), 2)


It is however ambiguous to use numpy's dtype that actually have the same sign and size (in that case on a 64bit machine)

In [25]:
code = '''
#pythran export ambiguous(int)
#pythran export ambiguous(int64)
def ambiguous(x): return x
'''
try:
    pythran.compile_pythrancode('dummy_module_name', code)
except pythran.syntax.PythranSyntaxError as e:
    print(e)

<unknown>:2:17 error: Ambiguous overloads
	ambiguous(int64)
	ambiguous(int).


And in case of invalid argument types, each overload is printed, as well as some information about the call site.

In [26]:
%%pythran
#pythran export some(float32)
#pythran export some(int)
def some(x): return x

In [27]:
try:
    some(True)
except TypeError as e:
    print(e)

Invalid call to pythranized function `some(bool)'
Candidates are:

    - some(int)
    - some(float32)



Overloads are useful to handle function with default parameters.

In [28]:
%%pythran
# pythran export func(int, str, float64)
# pythran export func(int, str)
# pythran export func(int, None, float64)
# pythran export func(int, None)
# pythran export func(int)
# pythran export func()

def func(a=1, b=None, c=1.0):
    return a, b, c

In [29]:
func(1, "hello", 2.)

(1, 'hello', 2.0)

In [30]:
func(1)

(1, None, 1.0)

It's possible to declare multiple entires in the same ``pythran export`` line

In [31]:
%%pythran
#pythran export foo(int), foo(str)
def foo(s): return s

In [32]:
foo(1), foo('1')

(1, '1')

The pythran export can also be used to export a global variable. But the global variable is not going to be shared, consider it as a read only view!

In [33]:
%%pythran
# pythran export thing
thing = 'stuff that matter'

In [34]:
thing

'stuff that matter'

It's also possible to ask pythran to export raw function pointer, using the ``capsule`` keyword.

In [35]:
%%pythran
#pythran export capsule corp(int, float)
def corp(x, y):
    return x + y

In [36]:
str(corp)[:40] + '...'

'<capsule object "corp(int, float)" at 0x...'

Pythran accepts pointer type, but it's only meaningful inside a capsule

In [37]:
%%pythran
#pythran export capsule corp(int*, int)
def corp(data, size):
    return data[size/2]

A Pythran function can take a capsule as input, using function type signatures.

In [38]:
%%pythran
#pythran export higher_order(int(int), int)
def higher_order(f, val):
    return f(val)
#pythran export capsule dummy(int)
def dummy(n):
    return n + 1

In [39]:
higher_order(dummy, 3)

4

Numpy arrays have a restriction on the supported data types: you cannot make arrays of string, sets etc!

In [40]:
code = '''
#pythran export invalid(str[])
def invalid(x): return x
'''
try:
    pythran.compile_pythrancode('dummy_module_name', code)
except pythran.syntax.PythranSyntaxError as e:
    print(e)

<unknown>:2:28 error: Unexpected token `[` at that point.


Pythran tries its best to provide detailed type information about parameters in case of mismatch

In [41]:
%%pythran
#pythran export basic(float32)
def basic(x): return x

In [42]:
try:
    import numpy as np
    x = np.arange(10)[::2]
    basic(x)
except TypeError as e:
    print(e)

Invalid call to pythranized function `basic(int64[:] (is a view))'
Candidates are:

    - basic(float32)



Pythran supports views with new axis

In [43]:
%%pythran
#pythran export views(float64[:,:])
def views(x):
    return x.shape

In [44]:
x = np.ones(5)[:, None]
y = np.ones(5)[None, :]
views(x), views(y)

((5, 1), (1, 5))

When usure of an object type, just print it!

In [45]:
%%pythran
#pythran export dump_type((float, bool) list)
#pythran export dump_type((int, complex) list)
def dump_type(arg1):
    return str([(type(x[0]), type(x[1])) for x in arg1])

In [46]:
dump_type([(1., True)]), dump_type([(1, 1j)])

('[(float, bool)]', '[(int_, complex)]')